# Importing necessary modules

In [63]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import Imputer, LabelEncoder, OneHotEncoder, StandardScaler
from six.moves import cPickle as pickle

In [3]:
def load_image(folder):
    valid_images = [".jpg", ".gif", ".png", ".tga"]
    all_char = os.listdir(folder)
    imgs_data = []
    Y = []
    
    for current_char in all_char:
        char_dir = os.path.join(folder,current_char)
        
        for image_name in os.listdir(char_dir):
            image_dir = os.path.join(char_dir, image_name)
            ext = os.path.splitext(image_dir)[1]
            
            if ext.lower() not in valid_images:
                continue
                
            try:
                img = Image.open(image_dir)
                img = np.array(img.resize((28,28),Image.ANTIALIAS))
                img = img.reshape(28*28)/255
                imgs_data.append(img)
                Y.append(current_char)
                
            except:
                print("unable to fetch image")
            
    print(np.array(imgs_data).shape, np.array(Y).reshape(len(Y),1).shape)
    return np.array(imgs_data,dtype=np.float32).T, np.array(Y).reshape(1,len(Y))

In [4]:
X, Y = load_image("/home/apurvnit/datasets/not_mnist/notMNIST_small")

unable to fetch image
unable to fetch image
(18724, 784) (18724, 1)


In [6]:
print("mean =",np.mean(X))
print("standard deviation = ",np.std(X))

mean = 0.425364
standard deviation =  0.458622


In [7]:
X_ = X * 255


In [8]:
X_ = (X_ - 128)/128

# Normalization - 
Making dataset of approx. zero mean and 1 variance

In [10]:
print("mean of X_ = ", np.mean(X_))
print("standar deviation of X_" ,np.std(X_))

mean of X_ =  -0.152596
standar deviation of X_ 0.913662


In [23]:
def randomize(X_var, Y_var):
    X, Y = shuffle(X_var.T, Y_var.T, random_state=random.randint(0,9))
    print(X.T.shape,Y.T.shape)
    return X.T, Y.T

In [25]:
X_1, Y_1 = randomize(X_, Y)
print(X_1, Y_1)

(784, 18724) (1, 18724)
[[-1.        -1.        -1.        ..., -0.578125   0.9921875 -1.       ]
 [-1.        -1.        -1.        ...,  0.8125     0.9921875 -1.       ]
 [-1.        -1.        -1.        ...,  0.984375   0.9921875 -1.       ]
 ..., 
 [-1.        -1.         0.96875   ..., -1.         0.9921875 -1.       ]
 [-0.828125  -1.         0.9921875 ..., -1.         0.9921875 -1.       ]
 [-0.109375  -1.         0.28125   ..., -1.         0.9921875 -1.       ]] [['E' 'D' 'A' ..., 'E' 'H' 'G']]


In [77]:
def data_preprocessing(X, Y):
    
    Y_1 = LabelEncoder().fit_transform(Y.T)
    Y_1 = Y_1.reshape(1, Y.shape[1])
    onehot = OneHotEncoder(categorical_features = [1])
    Y = onehot.fit_transform(Y_1) 
    X_1, Y_1 = randomize(X,Y)
    X_train = X_1[:, 0:int(0.75*X_1.shape[1])]
    Y_train = Y_1[:, 0:int(0.75 * Y_1.shape[1])]
    X_test = X_1[:, int(0.75 * X_1.shape[1]):]
    Y_test = Y_1[:, int(0.75 * X_1.shape[1]):]
    
    
#     return X_train,Y_train,X_test,Y_test
    

In [78]:
# X_train,Y_train,X_test,Y_test = data_preprocessing(X_, Y)
data_preprocessing(X_, Y)

(784, 18724) (1, 18724)


/home/apurvnit/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Since we have made and classified data for our program. lets save it so that we dont have to do such a long work again and again

In [79]:
def save_dataset(pickle_file):

    try:
      f = open(pickle_file, 'wb')
      save = {
        'train_dataset': X_train,
        'train_labels': Y_train,
        'test_dataset': X_test,
        'test_labels': Y_test,
        }
      pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
      f.close()
    except Exception as e:
      print('Unable to save data to', pickle_file, ':', e)
      raise

In [81]:
pickle_file = os.path.join("/home/apurvnit/Projects/My_deep-learning_examples/char_recognition", 'char_recognition.pickle')
save_dataset(pickle_file)
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

Compressed pickle size: 58868654
